In [1]:
from helper import export_projects, dataset_metadata, _export, download_datasets, upload_dataset, _export, _convert_dictkeys_to_snake
import datarobot as dr
import shutil
import pandas as pd
import os
import json
#Note

In [3]:
#Need to manually create this; because our codespace will NOT create an empty directory during init.
#Try to create a one-word .gitkeep and see if we can create this dir for them.
DIR = "migrations"
client = dr.Client()
me = client.get("account/info/").json()



## 1. 既存環境のプロジェクト・データセットをエキスポート

In [4]:
#MissingSchemaエラーがあったら環境変数が未設定の可能性大
original_projects = export_projects()
original_catalog_metadata = dataset_metadata(original_projects)

In [5]:
print("合計プロジェクト：", len(original_projects))
print("カタログIDのあるプロジェクト：", len([p for p in original_projects if p["catalogId"] is not None]))
print("-----------------------")

pd.DataFrame.from_records(original_projects)[["projectName","id","target","created"]]


合計プロジェクト： 82
カタログIDのあるプロジェクト： 51
-----------------------


,projectName,id,target,created
0,与信リスクのデータ_train_adv_options.xlsx,66ac813ddfabd5b3c254f0c7,貸し倒れ,2024-08-02T06:48:39.628614Z
1,週次売上予測データセット.xlsx,66ac7fd29d62d870a9785e70,今週の売上,2024-08-02T06:42:38.549474Z
2,skin_lesion.zip,66ab2bbb9b4a61619f66e5d2,None,2024-08-01T06:31:34.480563Z
3,与信リスクのデータ_train.xlsx,66a331cd6796f9292f7859aa,貸し倒れ,2024-07-26T05:19:21.748714Z
4,fc-reactic-gb Project [4ed8432],6699f8d6dfd7e0945fe91216,Sales (actual),2024-07-19T05:25:55.877390Z
...,...,...,...,...
77,DRDemo_PR_ContinuousAI_NYC311_00-OLD,6189bb39677d66f1e3a57bc6,DURATION,2021-11-09T00:05:13.432620Z
78,Utah House Pricing - Demo - v2,5fff1dc1c50bef9d7aa46c4b,price,2021-01-13T16:20:32.109249Z
79,MLDev Demo - Loans,5fdb79f2597db9c2478cfcd1,BadLoan,2020-12-17T15:32:18.620212Z
80,Enri,5fcfdf1cb39bd5561be4d7a7,Readmitted,2020-12-08T20:16:43.945798Z


## 2. 含みたいプロジェクトIDを以下のリストにて定義する

In [6]:
# list_of_project_ids = ["66ac7fd29d62d870a9785e70","66ac813ddfabd5b3c254f0c7","66ab2bbb9b4a61619f66e5d2", "6699c1882f700a81438cae3f", "66a331cd6796f9292f7859aa","6699c1152f700a81438cadf2"]
list_of_project_ids = ["66ac7fd29d62d870a9785e70"]
proj_df = pd.DataFrame(original_projects)
filtered_df = proj_df.query("id in @list_of_project_ids")

In [79]:
##移行対象アイテムのメタデータをファイルに書き出して保存場合

# with open(f"{DIR}/original-projects.json", "w") as f:
#     json.dump(original_projects, f, ensure_ascii=False, indent=4)
# original_catalog_metadata.to_csv(f"{DIR}/original-datasets.csv", index=False)

## 3. 次のバッチにフィルターして、Codespaceの空き容量を確認

In [7]:
target_datasets = filtered_df.fileName.unique()
original_catalog_metadata.query("name in @target_datasets")

,catalogId,name,datasetSize(MB),rowCount,columnCount,creationDate,createdBy,isLatestVersion,processingState,categories
1,66ac7f85825cacc9847542a1,週次売上予測データセット.xlsx.csv,2,10626,29,2024-08-02T06:41:09.956000Z,Greig Bloom,True,COMPLETED,"[TRAINING, PREDICTION]"


In [8]:
#ストレージの空き容量は20GBあるため、１バッチあたり20GB以下にする必要ある

next_batch_df = original_catalog_metadata.query("name in @target_datasets")
print(next_batch_df["name"].values)
print(next_batch_df["datasetSize(MB)"].sum())

['週次売上予測データセット.xlsx.csv']
2


#### ストレージが次回ダウンロード分に対して十分な空き容量かの確認

In [9]:
from IPython.core.display import HTML

raw = shutil.disk_usage(".")
free_mb = int(raw.free / (1_000_000))
print("空き容量: ", free_mb , "MB")

storage_ok = '<span style="color:green; font-weight:800">True</span>' if next_batch_df["datasetSize(MB)"].sum() < free_mb else '<span style="color:#f54c68; fontweight:800">False</span>'

print("次回移行分に必要な空き容量確保できる：：") 
HTML(storage_ok)


空き容量:  20822 MB
次回移行分に必要な空き容量確保できる：：


In [10]:
downloaded_datasets = download_datasets(next_batch_df, DIR, 4)
#downloaded_datasets

All downloads completed


## 5. Codespaceのストレージにダウンロードしたデータセットを新規環境にアップロード

In [11]:
new_datasets = []
for cat in downloaded_datasets:
    record = {"name": cat["name"]}
    if cat["complete"]:
        resp = upload_dataset(cat, DIR=DIR)
        if "catalogId" in resp:
            record = record | resp
        new_datasets.append(record)

In [12]:
new_datasets

[{'name': '週次売上予測データセット.xlsx.csv',
  'statusId': 'c70adcb0-451c-4f75-b68e-f9b3967076bb',
  'catalogId': '66c2acb1d3db1436792bfa4b',
  'catalogVersionId': '66c2acb1d3db1436792bfa4c'}]

## 6. 新しくアップロードしたデータセットのCatalogIDをプロジェクトデータに反映

In [13]:
next_batch_projects = []
next_batch_datasets = list(next_batch_df.name)
target_projects = [p for p in original_projects if p["fileName"] in next_batch_datasets]

for idx, p in enumerate(target_projects):
    if p["stage"] != "modeling":
        continue
    matched_file = [ds for ds in new_datasets if ds["name"] == p["fileName"]]
    if len(matched_file) == 1:
        new_record = target_projects[idx].copy()
        new_record["catalogId"] = matched_file[0]["catalogId"]
        next_batch_projects.append(new_record)
print("次回処理予定のプロジェクト数: ", len(next_batch_projects))

次回処理予定のプロジェクト数:  1


#### Autopilot実行前にダウンロード
#### 完了するまで繰り返してprocessingStateを確認


In [15]:
#新規環境に登録したデータセットのメタデータを取得
new_catalog = dataset_metadata(next_batch_projects,env="TARGET")
#データセット登録完了しているかを確認
new_catalog[["name","processingState"]]

,name,processingState
0,週次売上予測データセット.xlsx.csv,COMPLETED


## 7. Project / Autopilot 実行開始

In [16]:
## RemoteDisconnectedやConnectionErrorが発生する場合がある。その場合再度実行して、正常に処理が始まります。

adv_option_filter = ["downsampledMinorityRows", "downsampledMajorityRows","responseCap"]
TARGET_WORKERS = 8

print("こちらにてプロジェクトの進捗をご確認ください。")
print(f'{os.environ.get("DATAROBOT_ENDPOINT")[:-7]}/manage-projects')
for proj in next_batch_projects:
    print("プロジェクト開始： ", proj["projectName"])
    dr.Context.use_case = None
    new_proj = dr.Project.create_from_dataset(
        dataset_id=proj["catalogId"],
        project_name=proj["projectName"],
        )
    
    if "datetimePartitionColumn" in proj["partition"]:
        partitioning_method = dr.DatetimePartitioningSpecification(
            datetime_partition_column=proj["partition"]["datetimePartitionColumn"],
        )
    elif proj["partition"]["cvMethod"] == "group":
        partitioning_method = dr.GroupCV(
            reps=proj["partition"]["reps"],
            holdout_pct=proj["paritition"]["holdoutPct"],
            partition_key_cols=proj["partition"]["partitionKeyCols"]
        )
    else:
        #特別な要件がある場合は調べておいてね！
        partitioning_method = None

        
    #TypeError: AdvancedOptions.__init__()のエラーがスローされたら該当属性をadv_option_filterに追加する
    adv_options = {k:v for (k,v) in proj["advancedOptions"].items() if k not in adv_option_filter}
    adv_options = _convert_dictkeys_to_snake(adv_options)
    if "primaryLocationColumn" in proj:
        adv_options["primary_location_column"] = proj["primaryLocationColumn"]
    advanced_options = dr.AdvancedOptions(**adv_options)
    
    try:
        new_proj = new_proj.analyze_and_model(
                    target=  proj["target"],
                    mode=    proj["autopilotMode"],
                    target_type=  proj["targetType"],
                    unsupervised_mode= proj["unsupervisedMode"],
                    unsupervised_type= proj["unsupervisedType"],
                    metric = proj["metric"],
                    positive_class = proj["positiveClass"],
                    advanced_options=advanced_options,                    	
            	    worker_count=TARGET_WORKERS,
                    partitioning_method=partitioning_method,
        )
    except dr.errors.ClientError as e:
        print("エラー発生：, ", e)

こちらにてプロジェクトの進捗をご確認ください。
https://app.jp.datarobot.com/manage-projects
プロジェクト開始：  週次売上予測データセット.xlsx


### 次回のバッチがある場合は移行済ファイルをStorageから削除した上

### ステップ2に戻り、次のバッチ範囲を決めて実行してください

In [22]:
for file in next_batch_datasets:
    os.remove(f"{DIR}/{file}")


In [44]:
exported_projects = export_projects(env="TARGET")

In [45]:
#compare_projects = ["66ac844694d3910805cc1e5b","66ac8480347f45d126f97188"]
for p in exported_projects:
 #   if p["id"] in compare_projects:
        print(json.dumps(p, ensure_ascii=False))

{"id": "66b1781558968a16fdf966eb", "projectName": "与信リスクのデータ_train.xlsx", "fileName": "与信リスクのデータ_train.xlsx.csv", "stage": "modeling", "autopilotMode": 3, "created": "2024-08-06T01:10:54.445907Z", "target": "貸し倒れ", "metric": "LogLoss", "partition": {"cvMethod": "stratified", "validationType": "CV", "holdoutPct": 20.0, "reps": 5, "useTimeSeries": null, "partitionKeyCols": null, "validationLevel": null, "trainingLevel": null, "datetimeCol": null, "holdoutLevel": null, "userPartitionCol": null, "cvHoldoutLevel": null, "validationPct": null}, "advancedOptions": {"weights": null, "blueprintThreshold": 3, "responseCap": false, "seed": null, "defaultMonotonicIncreasingFeaturelistId": null, "defaultMonotonicDecreasingFeaturelistId": null, "onlyIncludeMonotonicBlueprints": false, "shapOnlyMode": false, "runLeakageRemovedFeatureList": true, "smartDownsampled": false, "majorityDownsamplingRate": null, "downsampledMinorityRows": null, "downsampledMajorityRows": null, "blendBestModels": false, "p